In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from Levenshtein import distance as lev_dist
from rdkit.ML.Cluster.Butina import ClusterData
from tensorflow import keras
from keras import layers
from keras.metrics import AUC, Accuracy, Precision
from keras.models import Model

from helper_functions import *

f:\anaconda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
f:\anaconda\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
f:\anaconda\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
model1 = tf.keras.models.load_model('Classification_model\model_crossval1.tf' ,custom_objects ={'Precision': Precision, 'AUC':AUC, "accuracy": Accuracy})
model2 = tf.keras.models.load_model('Classification_model\model_crossval2.tf',custom_objects ={'Precision': Precision, 'AUC':AUC, "accuracy": Accuracy})
model3 = tf.keras.models.load_model('Classification_model\model_crossval3.tf',custom_objects ={'Precision': Precision, 'AUC':AUC, "accuracy": Accuracy})
model4 = tf.keras.models.load_model('Classification_model\model_crossval4.tf',custom_objects ={'Precision': Precision, 'AUC':AUC, "accuracy": Accuracy})
model5 = tf.keras.models.load_model('Classification_model\model_crossval5.tf',custom_objects ={'Precision': Precision, 'AUC':AUC, "accuracy": Accuracy})

In [3]:
df_antiox = pd.read_csv("data\Classification_data.csv")

In [4]:
df_antiox.head()

,Unnamed: 0,Unnamed: 0.1,Sequence,Source,FRS,Chelator,partition
0,0,0,AHK,10.2174/138161207780414278,1,0,2
1,1,1,VHH,10.2174/138161207780414278,1,0,0
2,2,2,VHHANEN,10.2174/138161207780414278,1,0,1
3,3,3,VNPHDHQN,10.1021/jf00051a004,1,0,4
4,4,4,LVNPHDHQN,10.1021/jf00051a004,1,0,4


In [5]:
df_generate = pd.read_csv("data\Filter_Gen_Data.csv")

In [6]:
df_generate.head()

,Unnamed: 0,Filter_Gen_Data
0,0,PHKLP
1,1,RHANF
2,2,PYFNKL
3,3,LTVLA
4,4,SRYPSK


In [7]:
df_generate["Filter_Gen_Data"][0]

'PHKLP'

In [8]:
m1 = model1.predict(df_generate["Filter_Gen_Data"])
m2 = model2.predict(df_generate["Filter_Gen_Data"])
m3 = model3.predict(df_generate["Filter_Gen_Data"])
m4 = model4.predict(df_generate["Filter_Gen_Data"])
m5 = model5.predict(df_generate["Filter_Gen_Data"])

935/935 [==============================] - 4s 4ms/step


In [10]:
model1_lst, perc_model1 = generate_pep_seq(m1, 0.99, df_generate["Filter_Gen_Data"])
model2_lst, perc_model2 = generate_pep_seq(m2, 0.99, df_generate["Filter_Gen_Data"])
model3_lst, perc_model3 = generate_pep_seq(m3, 0.99, df_generate["Filter_Gen_Data"])
model4_lst, perc_model4 = generate_pep_seq(m4, 0.99, df_generate["Filter_Gen_Data"])
model5_lst, perc_model5 = generate_pep_seq(m5, 0.99, df_generate["Filter_Gen_Data"])

In [11]:
t1 = set(model1_lst).intersection(model2_lst)
t2 = set(t1).intersection(model3_lst)
t3 = set(t2).intersection(model4_lst)
t4 = list(set(t3).intersection(model5_lst))

In [12]:
len(t4)

124

In [13]:
df_peptidedb = pd.read_csv("data\dump.csv")

In [14]:
df_peptidedb.head(1)

,idpeptide,sequence,activity
0,1,FNKLKQGSSKRTCAKCFRKIMPSVHELDERRRGANRWAAGFRKCVS...,NaN


In [15]:
repeated = set(t4).intersection(df_peptidedb["sequence"])

In [16]:
repeated

{'WYW', 'YHYHH'}

In [17]:
t4.remove("WYW")
t4.remove('YHYHH')

In [18]:
len(t4)

122

In [19]:
ofile = open("122_gen_pep_fasta.txt", "w")

for i in range(len(t4)):

    ofile.write(">" + str(i+1)+"|" + "\n" +t4[i] + "\n")


ofile.close()

In [20]:
ofile = open("122_gen_pep_text.txt", "w")

for i in range(len(t4)):

    ofile.write(t4[i] + "\n")


ofile.close()

In [21]:
"""After the part above, all the 122 peptide sequences were check for their toxicity activity 
    by two different servers. One was ToxinPred and the second was ToxIBTL. 
    The intersection of both methods return 76 peptides.
"""
not_toxic =[]
df_toxic = pd.read_excel("data\ToxIBTL-pep.xlsx")
for i, seq in enumerate(df_toxic["Peptide"]):
    if df_toxic["Sequence label"][i] == 0:
        not_toxic.append(df_toxic["Peptide"][i])

In [22]:
nn, nn_seq, filterd_nn_seqs= [], [], []

for i, seq in enumerate(not_toxic):
    n, n_seq, _ = find_seqNN(seq, df_antiox)
    if n>=2:
        nn.append(n)
        nn_seq.append(n_seq)
        filterd_nn_seqs.append(seq)

In [23]:
cluster = ClusterData(filterd_nn_seqs, len(filterd_nn_seqs), 2, distFunc=lev_dist)

In [24]:
len(cluster)

12

In [25]:
idx = []
for clu in cluster:
    idx.append(clu[0])

In [26]:
idx

[52, 25, 0, 9, 6, 27, 2, 40, 21, 50, 23, 1]

In [27]:
final_peptides = [not_toxic[i] for i in idx]
print(final_peptides)

['PHYH', 'AHHHW', 'YWHW', 'LHYYMW', 'LYYYH', 'GYYWH', 'YYHPF', 'HWHYL', 'YWYHF', 'PHYY', 'LHHWY', 'YQYYW']


In [29]:
Antiox_filter_cluster = pd.DataFrame(final_peptides, columns=["peptide_seq"])
Antiox_filter_cluster["ID"] = idx
Antiox_filter_cluster["NN"] = Antiox_filter_cluster["ID"].map(lambda x: nn[x])
Antiox_filter_cluster["nearest_data"] = Antiox_filter_cluster["ID"].map(lambda x: nn_seq[x])
Antiox_filter_cluster["SMILES"] = Antiox_filter_cluster.peptide_seq.map(seq_to_smiles)
Antiox_filter_cluster["HydroMoment"] = Antiox_filter_cluster.peptide_seq.map(hydr_moment)
Antiox_filter_cluster["charge"] = Antiox_filter_cluster["peptide_seq"].map(lambda x: calculate_charge(x.upper()))
Antiox_filter_cluster["hydrophobicity"] = Antiox_filter_cluster["peptide_seq"].map(lambda x: assign_hydrophobicity(x.upper()))
Antiox_filter_cluster["av_hydrophobicity"] = Antiox_filter_cluster["hydrophobicity"].map(lambda x: sum(x)/len(x)) 
Antiox_filter_cluster["discrimination"] = Antiox_filter_cluster.apply(lambda x: calculate_discrimination(x.HydroMoment, x.charge), axis=1)

In [30]:
Antiox_filter_cluster

,peptide_seq,ID,NN,nearest_data,SMILES,HydroMoment,charge,hydrophobicity,av_hydrophobicity,discrimination
0,PHYH,52,2,HYH,O=C(O)[C@H](Cc1c[nH]cn1)NC(=O)[C@H](Cc1ccc(O)c...,0.056492,0,"[0.72, 0.13, 0.96, 0.13]",0.4850,0.053329
1,AHHHW,25,2,YYH,C[C@H](N)C(=O)N[C@@H](Cc1c[nH]cn1)C(=O)N[C@@H]...,0.168938,0,"[0.31, 0.13, 0.13, 0.13, 2.25]",0.5900,0.159477
2,YWHW,0,2,LHW,N[C@@H](Cc1ccc(O)cc1)C(=O)N[C@@H](Cc1c[nH]c2cc...,0.136834,0,"[0.96, 2.25, 0.13, 2.25]",1.3975,0.129171
3,LHYYMW,9,2,HHW,CSCC[C@H](NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)[C@H]...,0.084428,0,"[1.7, 0.13, 0.96, 0.96, 1.23, 2.25]",1.2050,0.079700
4,LYYYH,6,2,HYY,CC(C)C[C@H](N)C(=O)N[C@@H](Cc1ccc(O)cc1)C(=O)N...,0.067254,0,"[1.7, 0.96, 0.96, 0.96, 0.13]",0.9420,0.063488
5,GYYWH,27,2,RHH,NCC(=O)N[C@@H](Cc1ccc(O)cc1)C(=O)N[C@@H](Cc1cc...,0.112985,0,"[0.0, 0.96, 0.96, 2.25, 0.13]",0.8600,0.106658
6,YYHPF,2,2,YYH,N[C@@H](Cc1ccc(O)cc1)C(=O)N[C@@H](Cc1ccc(O)cc1...,0.205063,0,"[0.96, 0.96, 0.13, 0.72, 1.79]",0.9120,0.193580
7,HWHYL,40,2,LHG,CC(C)C[C@H](NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)[C@...,0.177450,0,"[0.13, 2.25, 0.13, 0.96, 1.7]",1.0340,0.167513
8,YWYHF,21,2,YYH,N[C@@H](Cc1ccc(O)cc1)C(=O)N[C@@H](Cc1c[nH]c2cc...,0.222983,0,"[0.96, 2.25, 0.96, 0.13, 1.79]",1.2180,0.210496
9,PHYY,50,3,YLYP,O=C(O)[C@H](Cc1ccc(O)cc1)NC(=O)[C@H](Cc1ccc(O)...,0.104547,0,"[0.72, 0.13, 0.96, 0.96]",0.6925,0.098692


In [31]:
Antiox_filter_cluster.to_csv("data/filter_cluster_data.csv")